# Verification of mortality effects

- Parameter variation `VerificationEFfects` in Anylogic
- 90 replicates per iteration or scenario

In [14]:
library(data.table)
library(xtable)
library(survival)
library(texreg)
library(fmsb)
library(metafor)
library(ggplot2)
library(simPH)
source("../src/utils.R")

In [15]:
# life table verification LE using mortality rates reported by CDC
mx = c(567.0,24.3 ,11.6 ,15.5 ,51.5 ,51.5 ,95.6 ,121.0 ,145.4 ,173.8 ,218.4,
    313.2 ,488.0 ,736.5 ,1050.2 ,1473.5 ,2206.9 ,3517.8 ,5871.7,13573.6)
mx = mx / 100000
le = lifetable(mx, ns = c(1, 4, rep(5, 2), 3, 2, rep(5, 14)))
le[1, "ex"]

[1] 79.02566

In [20]:
# read data files
m = fread("../output/data/mortality-testing-replicates.csv")
p = fread("../output/data/parameters-testing-replicates.csv")
e = fread("../output/data/environment-testing-replicates.csv")


In [4]:
head(p)

model_date,iteration,replicate,counties,people_per_county,fertility_adjustment,max_generation,mortality_cohort_size,income_mobility_cohort_size,measurement_cohort_window,...,mortality_fake_exp_coeff,smoking_rank_slope_exp_coeff,smoking_rank_slope_exp_coeff_se,smoking_income_coeff,smoking_county_income_exp_z,smoking_parent_smk_coeff,selected_generations,county_export_since_generation,recurrent_time_county_data,recurrent_heavy_computations
<chr>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
2022-02-14 at 17:15:03 CET,1,1,30,100,2.1,30,40,60,10,...,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 17:20:42 CET,10,1,30,100,2.1,30,40,60,10,...,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 17:14:54 CET,2,1,30,100,2.1,30,40,60,10,...,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 17:14:58 CET,3,1,30,100,2.1,30,40,60,10,...,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 17:17:06 CET,4,1,30,100,2.1,30,40,60,10,...,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10
2022-02-14 at 17:17:13 CET,5,1,30,100,2.1,30,40,60,10,...,0,0,0,"[-1.17434, -1.54797, -1.92578, -2.3151, -3.11795]",-0.2,0,"[20, 25, 30]",20,20,10


In [5]:
sp = varyingParameters(p, "model_date")
parameters = names(sp)[!names(sp) %in% c("iteration", "replicate")]
print(parameters)

character(0)


In [6]:
# redefine iteration and replicate indexes
# sp[, niteration := .GRP, by = parameters]
# sp[, nreplicate := 1:.N, by = niteration]
# np = sp[, c("iteration", "replicate", "niteration", "nreplicate", parameters), with = FALSE]
# m = merge(m, np, by = c("iteration", "replicate"))
# summary(np[niteration == 3, nreplicate])
# unique(np[, c("niteration", parameters), with = FALSE])

In [17]:
anyDuplicated(m[niteration == 1 & nreplicate == 1, id])
table(m[niteration == 1 & nreplicate == 1, generation])

ERROR: Error in .checkTypos(e, names_x): Object 'niteration' not found. Perhaps you intended iteration


In [15]:
grep("rank", names(m), value = TRUE)

[1] "rank"                         "parent_rank_difference"      
 [3] "county_rank_correlation"      "county_rank_slope"           
 [5] "county_rank_absolute"         "total_rank_slope_exposure"   
 [7] "rank_slope_exposure18"        "total_rank_absolute_exposure"
 [9] "rank_absolute_exposure18"     "total_rank_cor_exposure"     
[11] "rank_cor_exposure18"          "use_rank_slope"

In [7]:
summary(e[, nsi])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2936  0.3686  0.3860  0.3877  0.4084  0.4651 

In [8]:
mean(m[, age_death])

[1] 78.24359

# Exposure upward income mobility -> Mortality risk = 0.0, exposure until 18 years old

To explore correlation of the data produced by the ABM: 

- `total_rank_slope_exposure` and `rank_slope_exposure18`: exposure measures of place's rank-rank slopes (relative mobility)
- `income_group_mobility`: individual mobilityl, absolute value of (`income_group` - `parent_income_group`)
- `total_z_income_exposure`: exposure of standardized income

In [9]:
names(m)

[1] "iteration"                    "replicate"                   
 [3] "model_time"                   "id"                          
 [5] "fertility_control"            "parent_id"                   
 [7] "generation"                   "birthdate"                   
 [9] "age_death"                    "income_group"                
[11] "income"                       "parent_income_group"         
[13] "parent_income"                "rank"                        
[15] "parent_rank_difference"       "income_group_mobility"       
[17] "county"                       "nkids"                       
[19] "nmoves"                       "nmoves_kid"                  
[21] "smoker"                       "parent_smoker"               
[23] "county_mean_income"           "county_median_income"        
[25] "county_gini"                  "county_rank_correlation"     
[27] "county_rank_slope"            "county_rank_absolute"        
[29] "total_rank_slope_exposure"    "rank_slope_exposure18"       
[31] "total_rank_absolute_exposure" "rank_absolute_exposure18"    
[33] "total_rank_cor_exposure"      "rank_cor_exposure18"         
[35] "total_z_income_exposure"      "z_income_exposure30"

In [10]:
summary(m$income)
summary(m$total_income_exposure)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      0   28800   60000   86826  110800 1652000 

Length  Class   Mode 
     0   NULL   NULL 

In [21]:
t = copy(m)
t[, lincome := logIncome(income)]
t[, lcounty_income := logIncome(county_mean_income)]
t[, lparent_income := logIncome(parent_income)]
t[, status := 1] # there is no censoring
# t[, individual_income_mobility := abs(income_group_mobility)]
# t = t[age_death > 18]
mean(t$age_death)

[1] 78.29848

In [22]:
cor(t[, .(rank_absolute_exposure18, rank_slope_exposure18, county_rank_slope, county_rank_absolute, lincome, lparent_income,
    lcounty_income, age_death, total_z_income_exposure)])

,rank_absolute_exposure18,rank_slope_exposure18,county_rank_slope,county_rank_absolute,lincome,lparent_income,lcounty_income,age_death,total_z_income_exposure
rank_absolute_exposure18,1.00000000,-0.447482653,-0.055787111,0.35389663,0.158609656,0.196332579,0.32387867,0.023937684,0.461280728
rank_slope_exposure18,-0.44748265,1.000000000,0.096883887,-0.07654379,-0.008380642,0.008056660,-0.03091865,-0.002455141,-0.026652139
county_rank_slope,-0.05578711,0.096883887,1.000000000,-0.44120808,0.017974304,0.011624858,-0.01195654,0.001009842,-0.002434503
county_rank_absolute,0.35389663,-0.076543793,-0.441208076,1.00000000,0.271590324,0.118326915,0.65023408,0.030681208,0.520738557
lincome,0.15860966,-0.008380642,0.017974304,0.27159032,1.000000000,0.126265861,0.37511183,0.020452100,0.318068500
lparent_income,0.19633258,0.008056660,0.011624858,0.11832691,0.126265861,1.000000000,0.16054187,0.007870758,0.207996412
lcounty_income,0.32387867,-0.030918649,-0.011956538,0.65023408,0.375111834,0.160541866,1.00000000,0.052801851,0.813232882
age_death,0.02393768,-0.002455141,0.001009842,0.03068121,0.020452100,0.007870758,0.05280185,1.000000000,0.073633436
total_z_income_exposure,0.46128073,-0.026652139,-0.002434503,0.52073856,0.318068500,0.207996412,0.81323288,0.073633436,1.000000000


In [23]:
m1 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18, data =t)
m2 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group), data = t)
m3 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 +  as.factor(income_group) + lcounty_income, data = t)
m4 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group) + total_z_income_exposure, data = t)
cat(screenreg(list(m1, m2, m3, m4)))


                          Model 1         Model 2         Model 3         Model 4       
----------------------------------------------------------------------------------------
rank_absolute_exposure18       -0.65 ***       -0.47 ***       -0.24 **         0.21 ** 
                               (0.07)          (0.07)          (0.08)          (0.08)   
as.factor(income_group)2                       -0.03 *         -0.00           -0.00    
                                               (0.01)          (0.01)          (0.01)   
as.factor(income_group)3                       -0.04 ***       -0.01           -0.01    
                                               (0.01)          (0.01)          (0.01)   
as.factor(income_group)4                       -0.07 ***       -0.02           -0.01    
                                               (0.01)          (0.01)          (0.01)   
as.factor(income_group)5                       -0.11 ***       -0.02            0.02    
                    

In [24]:
m1 = coxph(Surv(age_death, status) ~ rank_slope_exposure18, data =t)
m2 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group), data = t)
m3 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group) + lcounty_income, data = t)
m4 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group) + total_z_income_exposure, data = t)
cat(screenreg(list(m1, m2, m3, m4)))


                          Model 1      Model 2         Model 3         Model 4       
-------------------------------------------------------------------------------------
rank_slope_exposure18           0.01         0.01           -0.01           -0.02    
                               (0.05)       (0.05)          (0.05)          (0.05)   
as.factor(income_group)2                    -0.03 **        -0.00           -0.00    
                                            (0.01)          (0.01)          (0.01)   
as.factor(income_group)3                    -0.05 ***       -0.02           -0.01    
                                            (0.01)          (0.01)          (0.01)   
as.factor(income_group)4                    -0.09 ***       -0.02           -0.01    
                                            (0.01)          (0.01)          (0.01)   
as.factor(income_group)5                    -0.12 ***       -0.02            0.02    
                                            (0.01)   

In [166]:
m1 = coxph(Surv(age_death, status) ~ county_rank_absolute, data =t)
m2 = coxph(Surv(age_death, status) ~ county_rank_absolute + as.factor(income_group), data = t)
m3 = coxph(Surv(age_death, status) ~ county_rank_absolute + as.factor(income_group) + lcounty_income, data = t)
m4 = coxph(Surv(age_death, status) ~ county_rank_absolute + as.factor(income_group) + total_z_income_exposure, data = t)
cat(screenreg(list(m1, m2, m3, m4)))


                          Model 1         Model 2         Model 3         Model 4       
----------------------------------------------------------------------------------------
county_rank_absolute           -0.86 ***       -0.73 ***       -0.17 ***        0.05    
                               (0.03)          (0.03)          (0.05)          (0.04)   
as.factor(income_group)2                       -0.01           -0.00           -0.00    
                                               (0.01)          (0.01)          (0.01)   
as.factor(income_group)3                       -0.01            0.01            0.01    
                                               (0.01)          (0.01)          (0.01)   
as.factor(income_group)4                       -0.01            0.01            0.01 *  
                                               (0.01)          (0.01)          (0.01)   
as.factor(income_group)5                       -0.07 ***       -0.01            0.03 ***
                    

In [23]:
snp = unique(np[, c("niteration", parameters), with = FALSE])

ERROR: Error in unique(np[, c("niteration", parameters), with = FALSE]): object 'np' not found


In [21]:
snp[use_rank_slope == FALSE & mortality_fake_exp_coeff == 0]

niteration,move_decision_rate,prob_move_random,use_rank_slope,weight_income_exp,mortality_fake_exp_coeff
<int>,<chr>,<dbl>,<lgl>,<chr>,<dbl>
1,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.01,FALSE,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
3,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.01,FALSE,"[0.8, 0.6, 0.4, 0.2, 0.05]",0
9,"[0.1, 0.1, 0.1, 0.1, 0.1]",0.01,FALSE,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
11,"[0.1, 0.1, 0.1, 0.1, 0.1]",0.01,FALSE,"[0.8, 0.6, 0.4, 0.2, 0.05]",0
13,"[0.1, 0.1, 0.1, 0.1, 0.1]",1.00,FALSE,"[0.0, 0.0, 0.0, 0.0, 0.0]",0
15,"[0.1, 0.1, 0.1, 0.1, 0.1]",1.00,FALSE,"[0.8, 0.6, 0.4, 0.2, 0.05]",0


In [22]:
m1 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group), data = t[niteration == 11])
m2 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group) +  total_z_income_exposure, data = t[niteration == 11])

cat(screenreg(list(m1, m2)))


                          Model 1          Model 2        
----------------------------------------------------------
rank_absolute_exposure18        -0.52 ***         0.10 ** 
                                (0.03)           (0.03)   
as.factor(income_group)2        -0.07 ***        -0.05 ***
                                (0.00)           (0.00)   
as.factor(income_group)3        -0.13 ***        -0.09 ***
                                (0.00)           (0.00)   
as.factor(income_group)4        -0.21 ***        -0.14 ***
                                (0.00)           (0.00)   
as.factor(income_group)5        -0.31 ***        -0.17 ***
                                (0.00)           (0.01)   
total_z_income_exposure                          -0.12 ***
                                                 (0.00)   
----------------------------------------------------------
AIC                       10867550.55      10865287.14    
R^2                              0.01             0.02 

In [38]:
iter = 11
age = 18
m1 = coxph( Surv(age_death, status)~ rank_absolute_exposure18, data = t[niteration==iter & age_death > age])
m2 = coxph( Surv(age_death, status)~ rank_absolute_exposure18 + as.factor(income_group), data = t[niteration==iter & age_death > age])
m3 = coxph( Surv(age_death, status)~ rank_absolute_exposure18 + as.factor(income_group) + total_z_income_exposure , data = t[niteration==iter & age_death > age])
cat(screenreg(list(m1, m2, m3)))


In [ ]:
ss = t[iteration == 11]

In [166]:
cor(ss[, .(rank_absolute_exposure18, rank_slope_exposure18, county_rank_slope, parent_rank_difference, income, 
    county_mean_income, age_death, total_z_income_exposure)])

,rank_absolute_exposure18,rank_slope_exposure18,county_rank_slope,parent_rank_difference,income,county_mean_income,age_death,total_z_income_exposure
rank_absolute_exposure18,1.00000000,-0.3994691726,-0.059017873,0.25567596,0.170610986,0.285309153,0.0441258970,0.46771502
rank_slope_exposure18,-0.39946917,1.0000000000,0.106261548,-0.01601226,-0.009811171,-0.005984632,0.0007771591,0.01616512
county_rank_slope,-0.05901787,0.1062615481,1.000000000,0.04421095,0.045096328,0.087250894,0.0040290261,0.06364535
parent_rank_difference,0.25567596,-0.0160122601,0.044210948,1.00000000,0.763426047,0.585414351,0.1532798293,0.51769038
income,0.17061099,-0.0098111713,0.045096328,0.76342605,1.000000000,0.496922841,0.0695300812,0.42385203
county_mean_income,0.28530915,-0.0059846318,0.087250894,0.58541435,0.496922841,1.000000000,0.0882564090,0.81485387
age_death,0.04412590,0.0007771591,0.004029026,0.15327983,0.069530081,0.088256409,1.0000000000,0.10628731
total_z_income_exposure,0.46771502,0.0161651199,0.063645349,0.51769038,0.423852029,0.814853866,0.1062873059,1.00000000


In [177]:
ggplot((ss[age_death > 0 & age_death < 110, .(rank_absolute_exposure18, age_death)]), aes(rank_absolute_exposure18, age_death)) +
    geom_point(size = 0.3) +
    geom_smooth(method = "loess") +
    theme_minimal()

`geom_smooth()` using formula 'y ~ x'



In [145]:
m1 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group), data = t[niteration == 8])
m2 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group) +  total_z_income_exposure, data = t[niteration == 8])

cat(screenreg(list(m1, m2)))


                          Model 1          Model 2        
----------------------------------------------------------
rank_absolute_exposure18        -1.38 ***        -0.73 ***
                                (0.02)           (0.05)   
as.factor(income_group)2        -0.05 ***        -0.05 ***
                                (0.00)           (0.00)   
as.factor(income_group)3        -0.11 ***        -0.11 ***
                                (0.00)           (0.00)   
as.factor(income_group)4        -0.16 ***        -0.16 ***
                                (0.00)           (0.00)   
as.factor(income_group)5        -0.21 ***        -0.20 ***
                                (0.00)           (0.00)   
total_z_income_exposure                          -0.06 ***
                                                 (0.00)   
----------------------------------------------------------
AIC                       11103369.22      11103123.66    
R^2                              0.02             0.02 

In [126]:
m1 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group), data = t[niteration == 9])
m2 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group) +  total_z_income_exposure, data = t[niteration == 9])

cat(screenreg(list(m1, m2)))


                          Model 1          Model 2        
----------------------------------------------------------
rank_absolute_exposure18        -0.23 ***         0.00    
                                (0.04)           (0.04)   
as.factor(income_group)2        -0.07 ***        -0.05 ***
                                (0.00)           (0.00)   
as.factor(income_group)3        -0.13 ***        -0.09 ***
                                (0.00)           (0.00)   
as.factor(income_group)4        -0.20 ***        -0.14 ***
                                (0.00)           (0.00)   
as.factor(income_group)5        -0.31 ***        -0.18 ***
                                (0.00)           (0.01)   
total_z_income_exposure                          -0.11 ***
                                                 (0.00)   
----------------------------------------------------------
AIC                       10851620.04      10849278.91    
R^2                              0.01             0.02 

In [127]:
m1 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group), data = t[niteration == 11])
m2 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group) +  total_z_income_exposure, data = t[niteration == 11])

cat(screenreg(list(m1, m2)))


                          Model 1          Model 2        
----------------------------------------------------------
rank_slope_exposure18           -0.04 *          -0.01    
                                (0.02)           (0.02)   
as.factor(income_group)2        -0.08 ***        -0.04 ***
                                (0.00)           (0.00)   
as.factor(income_group)3        -0.15 ***        -0.09 ***
                                (0.00)           (0.00)   
as.factor(income_group)4        -0.23 ***        -0.13 ***
                                (0.00)           (0.00)   
as.factor(income_group)5        -0.33 ***        -0.17 ***
                                (0.00)           (0.01)   
total_z_income_exposure                          -0.12 ***
                                                 (0.00)   
----------------------------------------------------------
AIC                       10867827.75      10865294.78    
R^2                              0.01             0.02 

In [97]:
m1 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group), data = t[iteration == 3])
m2 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group) +  total_z_income_exposure, data = t[iteration == 3])

cat(screenreg(list(m1, m2)))


                          Model 1          Model 2        
----------------------------------------------------------
rank_absolute_exposure18        -1.18 ***         0.26 ***
                                (0.03)           (0.04)   
as.factor(income_group)2        -0.06 ***        -0.06 ***
                                (0.00)           (0.00)   
as.factor(income_group)3        -0.12 ***        -0.10 ***
                                (0.00)           (0.00)   
as.factor(income_group)4        -0.18 ***        -0.15 ***
                                (0.00)           (0.00)   
as.factor(income_group)5        -0.24 ***        -0.20 ***
                                (0.00)           (0.00)   
total_z_income_exposure                          -0.12 ***
                                                 (0.00)   
----------------------------------------------------------
AIC                       10849816.95      10847781.59    
R^2                              0.01             0.02 

In [98]:
m1 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group), data = t[iteration == 4])
m2 = coxph(Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group) +  total_z_income_exposure, data = t[iteration == 4])

cat(screenreg(list(m1, m2)))


                          Model 1          Model 2        
----------------------------------------------------------
rank_absolute_exposure18        -1.06 ***        -0.01    
                                (0.03)           (0.04)   
as.factor(income_group)2        -0.05 ***        -0.04 ***
                                (0.00)           (0.00)   
as.factor(income_group)3        -0.10 ***        -0.10 ***
                                (0.00)           (0.00)   
as.factor(income_group)4        -0.16 ***        -0.16 ***
                                (0.00)           (0.00)   
as.factor(income_group)5        -0.21 ***        -0.20 ***
                                (0.00)           (0.00)   
total_z_income_exposure                          -0.11 ***
                                                 (0.00)   
----------------------------------------------------------
AIC                       10837194.40      10835440.56    
R^2                              0.01             0.01 

In [99]:
m1 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group), data = t[iteration == 5])
m2 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group) +  total_z_income_exposure, data = t[iteration == 5])

cat(screenreg(list(m1, m2)))


                          Model 1          Model 2        
----------------------------------------------------------
rank_slope_exposure18            0.03            -0.00    
                                (0.02)           (0.02)   
as.factor(income_group)2        -0.07 ***        -0.05 ***
                                (0.00)           (0.00)   
as.factor(income_group)3        -0.13 ***        -0.09 ***
                                (0.00)           (0.00)   
as.factor(income_group)4        -0.20 ***        -0.14 ***
                                (0.00)           (0.00)   
as.factor(income_group)5        -0.30 ***        -0.17 ***
                                (0.00)           (0.01)   
total_z_income_exposure                          -0.12 ***
                                                 (0.00)   
----------------------------------------------------------
AIC                       10847589.32      10845065.70    
R^2                              0.01             0.02 

In [100]:
m1 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group), data = t[iteration == 6])
m2 = coxph(Surv(age_death, status) ~ rank_slope_exposure18 + as.factor(income_group) +  total_z_income_exposure, data = t[iteration == 6])

cat(screenreg(list(m1, m2)))


                          Model 1          Model 2        
----------------------------------------------------------
rank_slope_exposure18           -0.02            -0.02    
                                (0.03)           (0.03)   
as.factor(income_group)2        -0.05 ***        -0.05 ***
                                (0.00)           (0.00)   
as.factor(income_group)3        -0.10 ***        -0.10 ***
                                (0.00)           (0.00)   
as.factor(income_group)4        -0.16 ***        -0.16 ***
                                (0.00)           (0.00)   
as.factor(income_group)5        -0.21 ***        -0.20 ***
                                (0.00)           (0.00)   
total_z_income_exposure                          -0.11 ***
                                                 (0.00)   
----------------------------------------------------------
AIC                       10832460.23      10831911.23    
R^2                              0.01             0.01 

In [25]:
# average number of county changes (residential moves)
mean(t$nmoves)

[1] 0

In [26]:
names(m)

[1] "iteration"                    "replicate"                   
 [3] "model_time"                   "id"                          
 [5] "fertility_control"            "parent_id"                   
 [7] "generation"                   "birthdate"                   
 [9] "age_death"                    "income_group"                
[11] "income"                       "parent_income_group"         
[13] "parent_income"                "rank"                        
[15] "parent_rank_difference"       "income_group_mobility"       
[17] "county"                       "nkids"                       
[19] "nmoves"                       "nmoves_kid"                  
[21] "smoker"                       "parent_smoker"               
[23] "county_mean_income"           "county_median_income"        
[25] "county_gini"                  "county_rank_correlation"     
[27] "county_rank_slope"            "county_rank_absolute"        
[29] "total_rank_slope_exposure"    "rank_slope_exposure18"       
[31] "total_rank_absolute_exposure" "rank_absolute_exposure18"    
[33] "total_rank_cor_exposure"      "rank_cor_exposure18"         
[35] "total_z_income_exposure"      "z_income_exposure30"         
[37] "niteration"                   "nreplicate"                  
[39] "move_decision_rate"           "prob_move_random"            
[41] "use_rank_slope"

In [80]:
cor(t[iteration == 1, .(rank_absolute_exposure18, rank_slope_exposure18, county_rank_slope, parent_rank_difference, income, 
    county_mean_income, age_death, total_z_income_exposure)])

,rank_absolute_exposure18,rank_slope_exposure18,county_rank_slope,parent_rank_difference,income,county_mean_income,age_death,total_z_income_exposure
rank_absolute_exposure18,1.00000000,-0.58808107,-0.36494191,0.21648696,0.16437461,0.7961243,0.07363204,0.85209201
rank_slope_exposure18,-0.58808107,1.00000000,0.43158104,-0.06956797,-0.05183537,-0.2404439,-0.01857546,-0.23613630
county_rank_slope,-0.36494191,0.43158104,1.00000000,-0.06903061,-0.04624647,-0.2115122,-0.02231119,-0.22990051
parent_rank_difference,0.21648696,-0.06956797,-0.06903061,1.00000000,0.75027048,0.2315235,0.14755655,0.23260229
income,0.16437461,-0.05183537,-0.04624647,0.75027048,1.00000000,0.2028128,0.06229670,0.19530794
county_mean_income,0.79612431,-0.24044387,-0.21151215,0.23152347,0.20281282,1.0000000,0.08146770,0.92330384
age_death,0.07363204,-0.01857546,-0.02231119,0.14755655,0.06229670,0.0814677,1.00000000,0.08860652
total_z_income_exposure,0.85209201,-0.23613630,-0.22990051,0.23260229,0.19530794,0.9233038,0.08860652,1.00000000


In [28]:
# function to run metafor
coxModel = function(data, formulas, replicate_column = "nreplicate") {

    output = list()
    for (i in seq_along(formulas)) {
        yi = NULL
        sei = NULL
        for (j in unique(data[[replicate_column]])) {
            temp = copy(data[get(replicate_column) == j])
            model = coxph(formulas[[i]], data = temp)
            yi = c(yi, model$coefficients[1])
            sei = c(sei, sqrt(model$var[1]))
        }
        output[[i]] = metafor::rma(yi = yi, sei = sei)
    } 
    return(output)  
}


In [39]:
# survival model 
treatment = "rank_absolute_exposure18"
covariates = c("lincome", "as.factor(income_group)", "lcounty_income", "total_z_income_exposure", "parent_rank_difference", "smoker")
f = list()
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[c(1,6)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[2], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(1,3, 5)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(2,3)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(1,4)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[c(1,5)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[c(2:6)], collapse = "+")))

In [45]:
test = coxModel(t[age_death > 18], f)

In [46]:
length(f)
f[[8]]
test[[8]]


[1] 8

Surv(age_death, status) ~ rank_absolute_exposure18 + as.factor(income_group) + 
    lcounty_income + total_z_income_exposure + parent_rank_difference + 
    smoker


Random-Effects Model (k = 30; tau^2 estimator: REML)

tau^2 (estimated amount of total heterogeneity): 0 (SE = 0.0308)
tau (square root of estimated tau^2 value):      0
I^2 (total heterogeneity / total variability):   0.00%
H^2 (total variability / sampling variability):  1.00

Test for Heterogeneity:
Q(df = 29) = 19.5449, p-val = 0.9065

Model Results:

estimate      se    zval    pval   ci.lb   ci.ub 
  0.1624  0.0629  2.5814  0.0098  0.0391  0.2857  ** 

---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [ ]:
t = m[niteration == 2]
t[, lincome := logIncome(income)]
t[, lcounty_income := logIncome(county_mean_income)]
t[, status := 1] # there is no censoring
# t[, individual_income_mobility := abs(income_group_mobility)]
# t = t[age_death > 18]
mean(t$age_death)

treatment = "rank_slope_exposure18"
covariates = c("lincome", "as.factor(income_group)", "lcounty_income", "total_z_income_exposure", "parent_rank_difference", "smoker")
f = list()
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[c(1,6)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[2], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(1,3, 5)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(2,3)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(1,4)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[c(1,5)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[c(2:6)], collapse = "+")))

In [ ]:


models = list()
for (i in seq_along(f)) {
    print(paste0("::::::::: Running model ", i))
    models[[i]] = coxph(f[[i]], data = t)

}

[1] 0.2856531

In [59]:
summary(models[[1]])


Call:
coxph(formula = f[[i]], data = t)

  n= 9166, number of events= 9166 

                            coef exp(coef) se(coef)      z Pr(>|z|)  
rank_absolute_exposure18 -0.7082    0.4925   0.2857 -2.479   0.0132 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

                         exp(coef) exp(-coef) lower .95 upper .95
rank_absolute_exposure18    0.4925       2.03    0.2814    0.8621

Concordance= 0.51  (se = 0.004 )
Likelihood ratio test= 6.18  on 1 df,   p=0.01
Wald test            = 6.15  on 1 df,   p=0.01
Score (logrank) test = 6.15  on 1 df,   p=0.01


In [39]:
cat(screenreg(models))


                          Model 1      Model 2        Model 3        Model 4        Model 5        Model 6        Model 7        Model 8      
----------------------------------------------------------------------------------------------------------------------------------------------
rank_absolute_exposure18      -0.71 *      -0.59 *        -0.42          -0.27          -0.31           0.12          -0.36          -0.38    
                              (0.29)       (0.29)         (0.29)         (0.29)         (0.29)         (0.30)         (0.29)         (0.29)   
lincome                                    -0.03 ***                      0.01                         -0.01           0.01                   
                                           (0.01)                        (0.01)                        (0.01)         (0.01)                  
smokerTRUE                                  0.03                                                                                             

# Exposure relative income mobility -> Mortality risk = 0.0, exposure until 18 years old

In [84]:
t = m[niteration == 3]
t[, lincome := logIncome(income)]
t[, lcounty_income := logIncome(county_mean_income)]
t[, status := 1] # there is no censoring
# t[, individual_income_mobility := abs(income_group_mobility)]
mean(t$age_death)

[1] 78.36714

In [85]:
# survival model 
treatment = "rank_slope_exposure18"
covariates = c("lincome", "as.factor(income_group)", "lcounty_income", "total_z_income_exposure", "parent_rank_difference", "smoker")
f = list()
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[1], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[2], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(1,3, 5)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(2,3)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+", paste0(covariates[c(1,4)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[c(1,5)], collapse = "+")))
f[[length(f)+1]] = formula(paste0("Surv(age_death, status) ~ ", treatment, "+",  paste0(covariates[c(1,5, 3)], collapse = "+")))

models = list()
for (i in seq_along(f)) {
    print(paste0("::::::::: Running model ", i))
    models[[i]] = coxph(f[[i]], data = t)

}

[1] "::::::::: Running model 1"
[1] "::::::::: Running model 2"
[1] "::::::::: Running model 3"
[1] "::::::::: Running model 4"
[1] "::::::::: Running model 5"
[1] "::::::::: Running model 6"
[1] "::::::::: Running model 7"
[1] "::::::::: Running model 8"


In [86]:
cat(screenreg(models))


                          Model 1         Model 2         Model 3         Model 4         Model 5         Model 6         Model 7         Model 8       
--------------------------------------------------------------------------------------------------------------------------------------------------------
rank_slope_exposure18           0.17 ***        0.14 ***        0.10 ***        0.08 **         0.08 **         0.05            0.09 **         0.08 ** 
                               (0.03)          (0.03)          (0.03)          (0.03)          (0.03)          (0.03)          (0.03)          (0.03)   
lincome                                        -0.04 ***                        0.01 ***                       -0.02 ***        0.01 ***        0.01 ***
                                               (0.00)                          (0.00)                          (0.00)          (0.00)          (0.00)   
as.factor(income_group)2                                       -0.07 ***         